## TN-Models Workbook 

We will be playing around with tn-models and going through some examples in this sample workbook this library is built with some predefined defaults that exist integrate with django by default, the api can be extended for other frameworks as well. 


## Simplest implementation 

In this snippet we will be creating a very simple api client using just the defaults associated with the tn-models library by passing in the minimum required variables 

client: an api client e.g axios that is already configured 
baseUrl: this should just be the uri of the route you would like to use, it is appended to the baseurl of the client 
    | in theory you could also pass in a full url here instead of configuring it in the client 
models: an object containing shapes of your date types, these will be used for inputs and outputs
    - entity *: this is the only requiredkey of the models 
 



In [10]:
/**
 * 
 * run this for some common utilities to be available in the environemnt 
 * use let if you intend to reassign the variable
 * 
 */
import z from 'npm:zod@3.22.4'
import {createApi, } from 'npm:@thinknimble/tn-models@2.5.0'
import axios from 'npm:axios@0.21.1'


let baseUrl = "http://localhost:8000"
let res




In [81]:




const client = axios.create({
    baseURL: baseUrl,
})

let userShape = {
    id: z.string(),
    email: z.string().email(),
    firstName: z.string(),
    lastName: z.string(),

}

let usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
    }
})

console.log('list: \n',await usersApi.list())
console.log('retrieve: \n',await usersApi.retrieve('1eec99ef-541e-474b-b54b-8c4688982bb5'))



list: 
 {
  count: 1000,
  next: null,
  previous: null,
  results: [
    {
      id: "1eec99ef-541e-474b-b54b-8c4688982bb5",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "2a9080c7-4caa-476f-8450-842e4d2b47f0",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "3aad2cd4-edc8-4500-9b89-68295d151456",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "6ccfe792-0319-4e83-8918-f1b0be8213c5",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "909e4654-409b-4cae-a386-111eefda7825",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "934bc383-969b-4b09-a474-da1e7624a1a2",
      email: "jane@doe.com",
      firstName: "John",
      lastName: "Doe"
    },
    {
      id: "a39a37b3-be42-45fa-90e0-7d41491875f3",
      email: "jane@doe.com",
      first

## Extending the capabilities while still using the defaults 

The above example is a basic implementation of an api we have only passed in the minimum required fields with this we have get and delete method access to our api with the pre-defined list & retrieve endpoints. 

Building off our basic example we will also enable access to the predefined create, update and delete endpoints, extending our get only api to work with post, patch and delete. To do this we only need to provide one more key to our models 

models: ...
    - create: we can reuse the same shape as the entity or create a new one

In [88]:

let userCreateShape = {
    email: z.string().email(),
    firstName: z.string(),
    lastName: z.string(),

}

usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
        create: userCreateShape,
    }
})



console.log("create: \n", await usersApi.create({email: "some1@example.com", firstName:"some", lastName:"example"}))
// not implemented in mock server yet
usersApi.update("id", {email: ""})





Response to service call with identifier < create > did not match expected type,
 errors: ZodError: [
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "id"
    ],
    "message": "Required"
  },
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "email"
    ],
    "message": "Required"
  },
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "firstName"
    ],
    "message": "Required"
  },
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "lastName"
    ],
    "message": "Required"
  }
]
    at Object.get error (file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/zod/3.22.4/lib/types.js:43:31)
    at parseResponse (file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:259:14)
    at Object.from

## Additional options with defaults 

Continuing just with our defaults we can also add filtering & pagination to our api 



In [ ]:
usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
        create: userShape,
        extraFilters:{
            search: z.string(),
        }
    }
})
// not implemented in server yet
await usersApi.list({filters:{search: "john"}, pagination:{page:1}}).then(console.log).catch(console.error)

